In [185]:
# Разведывательный анализ данных (EDA)
# цели и задачи проекта:
# 1. Сформулировать предположения и гипотезы для дальнейшего построения модели.
# 2. Проверить качество данных и очистить их, если это необходимо.
# 3. Определиться с параметрами модели.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

# чтение файла данных
df = pd.read_csv('stud_math.xls')


def emissionFilter(column, data):
    data[column] = data[column].astype(str)
    try:
        perc25 = data[column].astype(str).quantile(0.25)
        perc75 = data[column].quantile(0.75)
        IQR = perc75 - perc25
        print(data[column].between(perc25 - 1.5*IQR, perc75 + 1.5*IQR))
    except Exception:
        print(Exception, column, type(column))
    return data


# переименовать столбец 'studytime, granular' в 'studytimeG'
df.rename(columns={'studytime, granular': 'studytimeG'}, inplace=True)

# получение строковых столбцов
# замена пустых значений и просмотр количества уникальных значений
strdf = df.select_dtypes(include=['object'])
for column in strdf.columns:
    df[column] = df[column].apply(lambda x: None if (
        pd.isnull(x)) or (x == 'nan') else x.strip())
    # display(pd.DataFrame(df[column].value_counts()))

# помотрим количество записей с NaN
# print(df.isnull().sum())

# все записи для числовыx столбцов содержащие NaN, заполняем значением медианы
df.fillna(df.median(), inplace=True)

# применяем фильтр на выбросы для числовыx столбцов
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# получение числовых столбцов
numdf = df.select_dtypes(include=numerics)
# df.info()
# фильтруем все выбросы
for column in numdf.columns:
    perc25 = df[column].quantile(0.25)
    perc75 = df[column].quantile(0.75)
    IQR = perc75 - perc25
    try:
        df = df.loc[df[column].between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)]
    except Exception:
        print(Exception, column, type(column))

# сброс индесков с учетом исключенных строк
df = df.reset_index()

# Корреляционный анализ
#sns.pairplot(df, kind = 'reg')

# корреляционный анализ количественных переменных
scoreCorr = df.corr()['score'].sort_values()
# оставляем в выборке только столбцы с корреляцией более 0.12
display(scoreCorr.loc[abs(scoreCorr) > 0.12].index)
# Максимальная корреляция с score наблюдается для 'Medu', 'absences', 'age', 'goout'


def get_boxplot(column):
    fig, ax = plt.subplots(figsize=(14, 4))
    sns.boxplot(x=column, y='score',
                data=df.loc[df.loc[:, column].isin(
                    df.loc[:, column].value_counts().index[:10])],
                ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()


def get_stat_dif(column):
    cols = df.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'score'],
                     df.loc[df.loc[:, column] == comb[1], 'score']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break


# анализ номинативных переменных
for col in strdf.columns:
    get_stat_dif(col)
    # get_boxplot(col)
#  статистически значимые различия для колонок найдены для 'school', 'sex', 'schoolsup'

# Итоговый вывод: в нашем случае важные переменные, которые, возможно, оказывают влияние на оценку, это:
# 'absences', 'age', 'goout', 'Medu', 'school', 'sex', 'schoolsup'
stud_math_model = df.loc[:, ['absences', 'age',
                             'goout', 'Medu', 'school', 'sex', 'schoolsup']]
stud_math_model.head(10)

Index(['absences', 'age', 'goout', 'Medu', 'score'], dtype='object')

Найдены статистически значимые различия для колонки school
Найдены статистически значимые различия для колонки sex
Найдены статистически значимые различия для колонки schoolsup


,absences,age,goout,Medu,school,sex,schoolsup
0,6.0,18,4.0,4.0,GP,F,yes
1,4.0,17,3.0,1.0,GP,F,no
2,2.0,15,2.0,4.0,GP,F,no
3,4.0,16,2.0,3.0,GP,F,no
4,10.0,16,2.0,4.0,GP,M,no
5,0.0,16,4.0,2.0,GP,M,no
6,0.0,15,2.0,3.0,GP,M,no
7,0.0,15,1.0,3.0,GP,M,no
8,0.0,15,3.0,4.0,GP,F,no
9,2.0,15,3.0,4.0,GP,M,no
